## Library import

In [3]:
from PIL import Image
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import os
from matplotlib import pyplot as plt
from torchvision import transforms
from torch.nn import functional
import numpy as np
import pandas as pd
from torchmetrics import CharErrorRate
from torchmetrics.functional import char_error_rate

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

Defining some hyperparameters

In [4]:
data_path  = "vk-made-ocr"
train_path  = "vk-made-ocr/train/train"
test_path = "vk-made-ocr/test/test"
labels_path = "vk-made-ocr/train_labels.csv"
split_frac = 0.20
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
img_shape = ()

## Dataset class
 - loading data from the directory
 - Calculating the image shape and symbols occuring in the dataset

In [5]:
test_ind = []

In [6]:

class ImageData(Dataset):
    def load_image(self, path):
        img = Image.open(path).convert('RGB')
        transform = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.RandomRotation(degrees=15),    
        transforms.ToTensor()
        ])
        transform1 = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor() 
        ])
        if self.testing == True:
            res = transform1(img)
        else:
            res = transform(img)
        res.to(device)
        return  res


    def __init__(self, data_path,  testing = False , labels_path = None):
        self.samples = []
        self.symbols = set()
        self.labels = None
        self.img_shape = None
        super(self.__class__, self).__init__()
        self.testing = testing
        if testing  == False:
            with open(labels_path, "r") as f:
                self.labels = pd.read_csv(f)
            for ind, label_row in self.labels.iterrows():
                file_path = data_path + "/" + str(label_row['Id'])
            #    print(f"processing {file_path}")
                image = file_path
                image_label = str(label_row['Expected'])
                self.symbols.update(list(image_label))
                q = (image, image_label)
                self.samples.append(q)
        else:
             for filename in os.listdir(data_path):
                test_ind.append(filename)
                file_path = data_path + "/" + filename
                image = file_path
                self.samples.append(image)
              
        if self.testing == True:
            self.img_shape = self.load_image(self.samples[0]).shape
        else:
            self.img_shape = self.load_image(self.samples[0][0]).shape


    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        if self.testing == True:
            return (self.load_image(self.samples[idx]))
        else:
            res = self.samples[idx]
            return (self.load_image(res[0]), res[1])

In [7]:
train_dataset = ImageData(train_path, False, labels_path)

In [8]:
test_dataset = ImageData(test_path, True)

In [9]:
img_shape = train_dataset.img_shape


In [10]:
symbols = set()
for x in train_dataset.symbols:
    symbols.update(list(str(x)))

In [11]:
for i in symbols:
    if i == '😂':
        print("was")

In [12]:
# adding - this is important and it should be first !!! or the length would be fitted with trash
classes= ['😂'] + list(symbols)
classes_num = len(classes)
print(classes_num)
classes

609


['😂',
 'Ы',
 'ら',
 '[',
 'थ',
 'θ',
 'g',
 'j',
 'に',
 'س',
 '】',
 'ü',
 'ß',
 '分',
 'S',
 'ื',
 'ع',
 'Y',
 '·',
 '、',
 '▁',
 '백',
 "'",
 '；',
 'Μ',
 '頭',
 '藍',
 'ê',
 'ª',
 '吳',
 '賈',
 '法',
 'Р',
 '郭',
 '王',
 '後',
 'מ',
 '姜',
 '‐',
 '3',
 '£',
 'Â',
 'щ',
 '三',
 '+',
 'そ',
 'ο',
 'ト',
 'Л',
 'โ',
 '٬',
 'く',
 '_',
 '田',
 'б',
 'Τ',
 '์',
 'เ',
 'Ｃ',
 'é',
 'Ν',
 'Η',
 'Z',
 '＃',
 'х',
 'è',
 'и',
 'В',
 'χ',
 '.',
 'พ',
 '′',
 'і',
 '”',
 '●',
 '↓',
 '?',
 'ж',
 '・',
 '²',
 'σ',
 '慧',
 '\xad',
 'け',
 '№',
 'ὖ',
 'ｈ',
 'Ο',
 'э',
 'リ',
 '∘',
 '｜',
 'い',
 'ヨ',
 'V',
 '↑',
 'ς',
 'ง',
 'ה',
 '府',
 'δ',
 'ά',
 '|',
 '™',
 'κ',
 '규',
 '็',
 '板',
 '्',
 '{',
 '★',
 'а',
 'á',
 'T',
 '場',
 'ἰ',
 '☎',
 'Ｒ',
 'ö',
 'ю',
 'y',
 '記',
 'ш',
 'м',
 'ф',
 'Е',
 'ا',
 '煙',
 '？',
 '$',
 'ъ',
 'た',
 '和',
 'Π',
 'ك',
 'ў',
 'k',
 'ї',
 '승',
 'さ',
 '세',
 '沙',
 'だ',
 '傳',
 'ё',
 'Q',
 'l',
 '=',
 'г',
 'д',
 'ì',
 'E',
 'Υ',
 '駅',
 '蘇',
 '四',
 '）',
 'と',
 'չ',
 'з',
 '甲',
 'ย',
 'a',
 '桓',
 '董',
 'あ',
 

In [13]:
idx2char = {k:v for k,v in enumerate(classes, start=0)}
char2idx = {v:k for k,v in idx2char.items()}
print(idx2char)
print(char2idx)

{0: '😂', 1: 'Ы', 2: 'ら', 3: '[', 4: 'थ', 5: 'θ', 6: 'g', 7: 'j', 8: 'に', 9: 'س', 10: '】', 11: 'ü', 12: 'ß', 13: '分', 14: 'S', 15: 'ื', 16: 'ع', 17: 'Y', 18: '·', 19: '、', 20: '▁', 21: '백', 22: "'", 23: '；', 24: 'Μ', 25: '頭', 26: '藍', 27: 'ê', 28: 'ª', 29: '吳', 30: '賈', 31: '法', 32: 'Р', 33: '郭', 34: '王', 35: '後', 36: 'מ', 37: '姜', 38: '‐', 39: '3', 40: '£', 41: 'Â', 42: 'щ', 43: '三', 44: '+', 45: 'そ', 46: 'ο', 47: 'ト', 48: 'Л', 49: 'โ', 50: '٬', 51: 'く', 52: '_', 53: '田', 54: 'б', 55: 'Τ', 56: '์', 57: 'เ', 58: 'Ｃ', 59: 'é', 60: 'Ν', 61: 'Η', 62: 'Z', 63: '＃', 64: 'х', 65: 'è', 66: 'и', 67: 'В', 68: 'χ', 69: '.', 70: 'พ', 71: '′', 72: 'і', 73: '”', 74: '●', 75: '↓', 76: '?', 77: 'ж', 78: '・', 79: '²', 80: 'σ', 81: '慧', 82: '\xad', 83: 'け', 84: '№', 85: 'ὖ', 86: 'ｈ', 87: 'Ο', 88: 'э', 89: 'リ', 90: '∘', 91: '｜', 92: 'い', 93: 'ヨ', 94: 'V', 95: '↑', 96: 'ς', 97: 'ง', 98: 'ה', 99: '府', 100: 'δ', 101: 'ά', 102: '|', 103: '™', 104: 'κ', 105: '규', 106: '็', 107: '板', 108: '्', 109: '{', 110: '

Some statistics

In [14]:
print(f"total number of samples = {len(train_dataset)}")
print(f"shape of each image is {img_shape}")
print(f"symbols to recognize are {classes}, \nwhich is in total {len(classes)} elements")

total number of samples = 276000
shape of each image is torch.Size([3, 224, 224])
symbols to recognize are ['😂', 'Ы', 'ら', '[', 'थ', 'θ', 'g', 'j', 'に', 'س', '】', 'ü', 'ß', '分', 'S', 'ื', 'ع', 'Y', '·', '、', '▁', '백', "'", '；', 'Μ', '頭', '藍', 'ê', 'ª', '吳', '賈', '法', 'Р', '郭', '王', '後', 'מ', '姜', '‐', '3', '£', 'Â', 'щ', '三', '+', 'そ', 'ο', 'ト', 'Л', 'โ', '٬', 'く', '_', '田', 'б', 'Τ', '์', 'เ', 'Ｃ', 'é', 'Ν', 'Η', 'Z', '＃', 'х', 'è', 'и', 'В', 'χ', '.', 'พ', '′', 'і', '”', '●', '↓', '?', 'ж', '・', '²', 'σ', '慧', '\xad', 'け', '№', 'ὖ', 'ｈ', 'Ο', 'э', 'リ', '∘', '｜', 'い', 'ヨ', 'V', '↑', 'ς', 'ง', 'ה', '府', 'δ', 'ά', '|', '™', 'κ', '규', '็', '板', '्', '{', '★', 'а', 'á', 'T', '場', 'ἰ', '☎', 'Ｒ', 'ö', 'ю', 'y', '記', 'ш', 'м', 'ф', 'Е', 'ا', '煙', '？', '$', 'ъ', 'た', '和', 'Π', 'ك', 'ў', 'k', 'ї', '승', 'さ', '세', '沙', 'だ', '傳', 'ё', 'Q', 'l', '=', 'г', 'д', 'ì', 'E', 'Υ', '駅', '蘇', '四', '）', 'と', 'չ', 'з', '甲', 'ย', 'a', '桓', '董', 'あ', 'W', 'ա', 'р', 'Ｍ', '\\', 'Ι', 'う', '－', 'ц', '宗', 'b', '原'

## Train test split
Here we split the dataset in train and test in the fraction defined before

In [15]:
# that line used to work with fracs locally but on kaggle it stopped so i added ints manually which is not nice but whatever
trainset, valset = random_split(train_dataset, [220800, 55200])
train_loader = DataLoader(trainset, batch_size=128, shuffle=True, num_workers=0)
val_loader = DataLoader(valset, batch_size=128, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size = 128, shuffle = False, num_workers = 0)

In [16]:
print(f"length of trainset is {len(trainset)}, length of validation set is {len(valset)}")
print(f"length of training set is {len(train_loader)}, length of validation loader {len(val_loader)}")

length of trainset is 220800, length of validation set is 55200
length of training set is 1725, length of validation loader 432


In [17]:
print(type(trainset[0][1]))

<class 'str'>


This nasty line of code below crashed my whole jupiter kernel I have no idea why :( It just made an infinity loop even though in collab it worked fine 

In [18]:
image_batch, text_batch= next(iter(train_loader))
print(image_batch.size(), text_batch)

torch.Size([128, 3, 224, 224]) ('3БЪІ', 'клубом', '2', 'гм!уз>д[Т№8', 'будучи', 'золотого', 'серьезно', 'что', 'Владимирской', 'идти', 'ISA', 'минувшую', '27.12.1948', 'а', 'Flower-memory', 'Но час настал,', 'Семей', 'утром.', 'корп', 'MIDDLE', 'CARE', 'садов,', 'передавались', 'В накошенные', 'И складывают', 'Солоха', 'мужской', 'ПРОШЕДШЕМ', 'костюм.', 'жёсткий', 'SUSHI/', 'влияния', 'Париж', 'произошло', 'QUESTA', '877', 'Шымкент', 'реки', '“”Лпмн«|4Шц', 'зиме.', 'RUyI', 'очень', '1178', 'поврежденный', 'стокс,', 'осуществления', 'ОБЕИХ', '023453', 'Lyon.', 'the', 'правом.', 'KickerClub.com', 'наше', 'слух', 'земной', 'ДАННЫМ,', 'абсолютно', 'Dunkelheit,', 'СЧЕТ', 'Гусев', 'опасной,', 'сарматский', '(84.1', 'СПАЛЬНОЕ', 'Лейб', '(Народная', 'казахстан', 'Область', 'DESSERT', 'всё мимо, мимо', 'готов', 'Й_6ДрФЛ`$)эП;к', 'W', 'женск', 'BEIJING DISTRICT HEATING GROUP CO.，LTD.', 'жил', 'орудия', 'психолога.', 'werden', '5268 7912 3456 7890', 'ТЕЧЕНИЕ', 'FIELD', 'легендар', 'послед', '(196

## Model class

Being honest I didn't create a bicycle, I tried at first but the quality really sucked so i just read the article the picture is taken from and a few articles from other cool guys and taken the structure from them, worked not bad even though the optimizator matters that's the fact

In [19]:
class MyNeuralNetwork(nn.Module):

    def __init__(self, vocab_size, dropout=0.5):
        super(MyNeuralNetwork, self).__init__()

        self.dropout = nn.Dropout(dropout)
        # convilutional part
        self.convlayer = nn.Sequential(
            nn.Conv2d(3, 32, (3,3), stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d((2,2), 2),

            nn.Conv2d(32, 64, (3,3), stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d((2,2), 2),

            nn.Conv2d(64, 128, (3,3), stride=1, padding=1),
            nn.ReLU(),

            nn.Conv2d(128, 256, (3,3), stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d((1,2), 2),

            nn.Conv2d(256, 512, (3,3), stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),

            nn.Conv2d(512, 512, (3,3), stride=1, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d((1,2), 2),

            nn.Conv2d(512, 512, (2,2), stride=1, padding=0),
            self.dropout
        )

        self.mapSeq = nn.Sequential(
            nn.Linear(6656, 2048),
            self.dropout,
            nn.Linear(2048, 512),
            self.dropout
        )
        #Bi Lstm (gru)
        
        self.lstm_0 = nn.GRU(512, 512, bidirectional=True)  
        self.lstm_1 = nn.GRU(1024, 512, bidirectional=True)

        self.out = nn.Sequential(
            nn.Linear(1024, vocab_size),
        )
        
        
    def forward(self, x): 
        
        x = self.convlayer(x)
        x = x.permute(0, 3, 1, 2)
        x = x.view(x.size(0), x.size(1), -1)
      #  print(x.shape)
        x = self.mapSeq(x)
        x, _ = self.lstm_0(x)
        x, _ = self.lstm_1(x)
        x = self.out(x)   
        x = x.permute(1, 0, 2)
        return x

## Loss function

Used CTCloss function just like in the article.
At first I tried to form a class for the loss with the method compute loss but got a value error during the forward operation and couldn't really figure out how to fix it except making it all into a big class, but i didn't find it fancy so i left it like that.

In [20]:
criterion = nn.CTCLoss(blank=0, zero_infinity=True)
def encode_text_batch(text):
    
    text_lens = torch.IntTensor([len(t) for t in text])
    
    text = "".join(text)
    text = torch.IntTensor([char2idx[c] for c in text])
    return text, text_lens

def compute_loss(y, pred):
    pred = functional.log_softmax(pred, 2)
    pred_lens = torch.full(size=(pred.size(1),), fill_value=pred.size(0), dtype=torch.int32).to(device) 
    y,y_lens = encode_text_batch(y)
    loss = criterion(pred, y, pred_lens, y_lens)
    return loss

At first I have been using models with a very small learning rate and they had been stucking at some points so I have chosen the bigger one. Also used a nesterov momentum and learning rate decay as an optimization for learning. Tried to use ADAM before but it did work awfully more probably cause i couldn't figure out the good parameters for that but sgd with optimizations works fine.

In [21]:
model = MyNeuralNetwork(classes_num)
model = model.to(device)
opt = torch.optim.SGD(
    model.parameters(), lr=0.02, nesterov=True,
    weight_decay=1e-5, momentum=0.7
)

The saint three: nn model, optimizator and loss functions are working fine together making a harmonious solution.Just some good old bad ass training, as u could see not many epoches nedeed to train it.

In [22]:
def train_model(model, train_loader, val_loader, opt, n_epochs: int):
    train_loss = []
    val_loss = []
    
    for epoch in range(n_epochs):
        print(f"processing {epoch} epoch")
        ep_train_loss = []
        ep_val_loss = []
        ep_val_accuracy = []

        model.train(True) 
        i  = 0
        for X_batch, y_batch in train_loader:
            i += 1
            if i % 100 == 0:
                print(f"processed  {i} batches")
            X_batch = X_batch.to(device)
            predictions = model.forward(X_batch)
            loss = compute_loss(y_batch, predictions) # in this sequence this is important
            loss.backward()
            opt.step()
            opt.zero_grad()
            ep_train_loss.append(loss.item())

        model.eval() 
        model.to(device)
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch =  X_batch.to(device)
                predictions = model(X_batch)
                loss = compute_loss(y_batch, predictions)
                ep_val_loss.append(loss.item())
                
        # print the results for this epoch:

        train_loss.append(np.mean(ep_train_loss))
        val_loss.append(np.mean(ep_val_loss))
        
        print(f"\t  training loss: {train_loss[-1]:.6f}")
        print(f"\tvalidation loss: {val_loss[-1]:.6f}")

    return train_loss , val_loss

In [23]:
train_loss, val_loss = train_model(model, train_loader, val_loader, opt, n_epochs=20 )

processing 0 epoch
processed  100 batches
processed  200 batches
processed  300 batches
processed  400 batches
processed  500 batches
processed  600 batches
processed  700 batches
processed  800 batches
processed  900 batches
processed  1000 batches
processed  1100 batches
processed  1200 batches
processed  1300 batches
processed  1400 batches
processed  1500 batches
processed  1600 batches
processed  1700 batches
	  training loss: 4.606745
	validation loss: 4.141559
processing 1 epoch
processed  100 batches
processed  200 batches
processed  300 batches
processed  400 batches
processed  500 batches
processed  600 batches
processed  700 batches
processed  800 batches
processed  900 batches
processed  1000 batches
processed  1100 batches
processed  1200 batches
processed  1300 batches
processed  1400 batches
processed  1500 batches
processed  1600 batches
processed  1700 batches
	  training loss: 3.998878
	validation loss: 3.956088
processing 2 epoch
processed  100 batches
processed  200

KeyboardInterrupt: 

In [ ]:
print(ep_train_loss)

Plotting the losses, looking for overfitting

In [ ]:
def plot_train_process(train_loss, val_loss):

    plt.plot(train_loss, label='train')
    plt.plot(val_loss, label='validation')
    plt.legend()

Hmm...Not that bad, could put even smaller number of epoches

In [ ]:
plot_train_process(train_loss, val_loss)

## Results

Now let's calculate metrics. I used CER cause i trust teachers who recommended it and I appreciate levenshtein distance for string difference

In [24]:
def decode_predictions(pred):
    pred = functional.softmax(pred, 2).argmax(2) 
    pred = pred.numpy().T 

    text_batch_tokens_new = []
    for text_tokens in pred:
        text = [idx2char[idx] for idx in text_tokens]
        text = "".join(text)
        text_batch_tokens_new.append(text)

    return text_batch_tokens_new

In [39]:
results_train = pd.DataFrame(columns=['actual', 'prediction', 'distance'])
#train_loader = DataLoader(trainset, batch_size=128, num_workers=1, shuffle=False)
cer_metric = []
with torch.no_grad():
    for image_batch, text_batch in val_loader:
        #make prediction
        pred = model(image_batch.to(device)) 
        text_batch_pred = [t.replace("😂", "") for t in decode_predictions(pred.cpu())]
         #print(text_batch, text_batch_pred)
        # calc metric 
        batch_dist = []
        for i in range(len(text_batch)):
            dist = char_error_rate(text_batch[i], text_batch_pred[i]).numpy()
            batch_dist.append(dist)
        cer_metric.append(batch_dist)
        #make a dataset of what was recognized and what actually should be 
        df = pd.DataFrame(columns=['actual', 'prediction', 'distance'])
        df['actual'] = text_batch
        df['prediction'] = text_batch_pred
        df['distance'] = batch_dist
        results_train = pd.concat([results_train, df])
results_train = results_train.reset_index(drop=True)

In [26]:
result_metric = char_error_rate(list(results_train['actual']),results_train['prediction'])
print(f"result CER for our OCR model is {result_metric}")

result CER for our OCR model is 0.7014426589012146


In [31]:
results_train = pd.DataFrame(columns=[ 'Predicted'])
test_loader = DataLoader(test_dataset, batch_size = 128, shuffle = False, num_workers = 0)
#train_loader = DataLoader(trainset, batch_size=128, num_workers=1, shuffle=False)
cer_metric = []
with torch.no_grad():
    for image_batch  in test_loader:
        #make prediction
        pred = model(image_batch.to(device)) 
        text_batch_pred = [t.replace("😂", "") for t in decode_predictions(pred.cpu())]
        #make a dataset of what was recognized and what actually should be 
        df =  pd.DataFrame(columns=['Predicted'])
        df['Predicted'] = text_batch_pred
        results_train = pd.concat([results_train, df])
results_train['Id'] = test_ind
results_train = results_train.reset_index(drop=True)

In [32]:
results_train['Id'] = test_ind
results_train = results_train.reset_index(drop=True)

In [ ]:
results_train = results_train.reset_index(drop=True)

In [33]:
results_train

,Predicted,Id
0,,31973.jpg
1,,30778.jpg
2,MULLEESSоLS,19812.jpg
3,,22735.jpg
4,Етеялса,38246.jpg
...,...,...
52994,nnEE,31745.jpg
52995,,49557.jpg
52996,сввалето6,11590.jpg
52997,2ка,49917.jpg


In [37]:
len(results_train)

52999

In [35]:
results_train.to_csv("submission4.csv", index = False)

In [ ]:
result_metric = char_error_rate(list(results_train['actual']),results_train['prediction'])
print(f"result CER for our OCR model is {result_metric}")

Let's take a look at the worst recognized ones

In [40]:
results_train.sort_values(by='distance', ascending=False)

,actual,prediction,distance
9721,РЕСПУБЛИКА,,inf
38494,цент,,inf
38425,и,,inf
9028,ли,,inf
38431,РОССИИ,,inf
...,...,...,...
33540,ON,ON,0.0
2476,по,по,0.0
33547,Honshu,Honshu,0.0
50625,брюхо.,брюхо.,0.0
